In [8]:
import numpy as np
import os
import pickle
from IMDBModel import IMDBModel
from embedding import Embedding
from keras.preprocessing import sequence
import time
from glove_utils import load_embedding
from data_utils import IMDBDataset
from pprint import pprint
from attacker import Attacker
from explainers import SBE, LIMEExplainer
from pos_taggers import TextBlobTagger, SpacyTagger
from display_utils import FixingResult, html_render, display_html
import utils

In [9]:
%load_ext autoreload
%autoreload 2

### Load IMDB data

In [10]:
# Load data
maxlen = 200
batch_size = 32
print('Loading data...')
(train_text, x_train, y_train), (test_text, x_test, y_test) = IMDBDataset.load_data()
x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding = 'pre', truncating = 'pre')
x_test = sequence.pad_sequences(x_test, maxlen=maxlen, padding = 'pre', truncating = 'pre')
print('Data loaded.')


Loading data...
Data loaded.


In [11]:
results_path = 'results/'

In [12]:
none_results_path = results_path + 'fix_fasttext_threshold_0.5_beam_4_none.pickle' # results using no explainer and targeting all words
lime_results_path = results_path +  'fix_fasttext_threshold_0.5_beam_4_lime.pickle' # results using LIME explainer
sbe_results_path = results_path + 'fix_fasttext_threshold_0.5_beam_4_sbe.pickle'   # results using SBE explainer

In [19]:
none_results_data = pickle.load(open(none_results_path, 'rb'))
lime_results_data = pickle.load(open(lime_results_path, 'rb'))
sbe_results_data = pickle.load(open(sbe_results_path, 'rb'))

In [20]:
none_fix_result = FixingResult(none_results_data)
lime_fix_result = FixingResult(lime_results_data)
sbe_fix_result  = FixingResult(sbe_results_data)

In [15]:
##### Using No explainer
print("Using no explainer: success_rate = %f , avg_nwords_changed = %f" % \
    (none_fix_result.get_success_rate(), none_fix_result.get_modification_percentage()))

print("Using LIME explainer: success_rate = %f, avg_nwords_changed = %f " % \
     (lime_fix_result.get_success_rate(), lime_fix_result.get_modification_percentage()))

print("Using SBE explainer: success_rate = %f, avg_nwords_changed = %f " % \
     (sbe_fix_result.get_success_rate(), sbe_fix_result.get_modification_percentage()))



Using no explainer: success_rate = 0.932000 , avg_nwords_changed = 0.016602
Using LIME explainer: success_rate = 0.766000, avg_nwords_changed = 0.011665 
Using SBE explainer: success_rate = 0.754000, avg_nwords_changed = 0.011531 


In [16]:
none_fix_result.visualize(max_display_count = 10, show_successful_only = True)

#0   index = 339
FIX SUCCEEDED
SUGGESTED REPLACEMENTS OPTION  0 : 
Original prediction: 0.439153


New prediction: 0.772530


SUGGESTED REPLACEMENTS OPTION  1 : 
Original prediction: 0.439153


New prediction: 0.700662


SUGGESTED REPLACEMENTS OPTION  2 : 
Original prediction: 0.439153


...

SUGGESTED REPLACEMENTS OPTION  2 : 
Original prediction: 0.692838


New prediction: 0.403471


SUGGESTED REPLACEMENTS OPTION  3 : 
Original prediction: 0.692838


New prediction: 0.430243


--------------------------------------------------


In [17]:
none_fix_result.original_texts[43]

', that entire trip is missing from the story , and so is most of what goes on at the theatrical rehearsals and most of what discomfits sir thomas on his return ; and in fact sir thomas himself , and his wife , are different from what they were . above all , the scene in which crawford proposes to fanny , one of austens most complex , is simplified to a bare telling ; missing from it is fannys staunch upholding of what she knows to be right , and what she knows to be very probably true , against all the distractions crawford and anybody else can throw at her . to a male reader , now no less than when it was written , it reveals with unmatched clarity - - unmatched , at any rate , as far as i know - - what a woman goes through in trying to deal fairly but firmly with a man she has reason to distrust . its a brilliant scene , in the novel ; on tv its just a scene . and , fanny , oh , fanny : when will we see your like ?'

In [21]:
none_fix_result.original_predictions[43]

0.9918126

In [22]:
none_fix_result.replacements[43]

[([(171, 'brilliant', 'grandiose'),
   (67, 'complex', 'convoluted'),
   (129, 'reveals', 'discloses'),
   (159, 'firmly', 'categorically'),
   (70, 'simplified', 'rationalized'),
   (131, 'unmatched', 'unprecedented'),
   (135, 'unmatched', 'unprecedented')],
  0.4240751),
 ([(171, 'brilliant', 'grandiose'),
   (67, 'complex', 'convoluted'),
   (129, 'reveals', 'discloses'),
   (159, 'firmly', 'categorically'),
   (70, 'simplified', 'rationalized'),
   (131, 'unmatched', 'unprecedented'),
   (189, 'oh', 'ah')],
  0.45181993),
 ([(171, 'brilliant', 'grandiose'),
   (67, 'complex', 'convoluted'),
   (129, 'reveals', 'discloses'),
   (159, 'firmly', 'categorically'),
   (70, 'simplified', 'rationalized'),
   (131, 'unmatched', 'unprecedented'),
   (16, 'goes', 'disappears')],
  0.45224956),
 ([(171, 'brilliant', 'grandiose'),
   (67, 'complex', 'convoluted'),
   (129, 'reveals', 'discloses'),
   (159, 'firmly', 'categorically'),
   (70, 'simplified', 'rationalized'),
   (131, 'unmatched'